In [6]:
# Please modify these with your paths, input / output files,  etc
code_path = '/data/software/SynthSR'  # location of SynthSR
visible_gpu_devices = '' # Leave it empty to use the CPU (it takes less than a minute anyway)
model_file = '/data/low_field/hyperfine/SynthSR_analysis/model_for_james.h5' # location of .h5 model file
T1 = 'T1_FILE' # input T1, resampled to 1mm
T2 = 'T2_FILE' # input T2, registered & resampled to T1
output_file = 'OUTPUT_FILE/t1_SR.nii.gz' # the output will be written here


In [2]:
import sys
sys.path.append(code_path)  

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"] =   visible_gpu_devices

import numpy as np
from ext.lab2im import utils
from ext.neuron import models as nrn_models
from ext.lab2im.edit_volumes import align_volume_to_ref 


Using TensorFlow backend.


In [3]:
unet_model = nrn_models.unet(nb_features=24,
                             input_shape=[256, 256, 256, 2],
                             nb_levels=5,
                             conv_size=3,
                             nb_labels=1,
                             feat_mult=2,
                             dilation_rate_mult=1,
                             nb_conv_per_level=2,
                             conv_dropout=False,
                             final_pred_activation='linear',
                             batch_norm=-1,
                             input_model=None)



In [4]:
print('loading', model_file)
unet_model.load_weights(model_file)

loading /tmp/model_for_james.h5


In [10]:
# read images
im1_orig, aff_orig, _ = utils.load_volume(T1,im_only=False,dtype='float')
im2_orig = utils.load_volume(T2,im_only=True,dtype='float')

# rearrange axis so aff is approximately diagonal
im1, aff = align_volume_to_ref(im1_orig, aff_orig, return_aff=True)
im2 = align_volume_to_ref(im2_orig, aff_orig)

# Normalization to [0,1] interval (please ignore the 3.0 and 2.0, it's a long story...)
minimum = np.min(im1)
im1 = im1 - minimum
spread = np.max(im1) / 3.0
im1 = im1 / spread 
im2 = im2 - np.min(im2)
im2 = im2 / np.max(im2) * 2.0

# Pad to 256
I = np.stack([im1, im2], axis=-1)[np.newaxis,...]
S = np.zeros([1, 256, 256, 256, 2]) 

i1 = np.floor((256-I.shape[1])/2).astype('int')
j1 = np.floor((256-I.shape[2])/2).astype('int')
k1 = np.floor((256-I.shape[3])/2).astype('int')
S[0, i1:i1+I.shape[1], j1:j1+I.shape[2], k1:k1+I.shape[3], :] = I

In [11]:
output = unet_model.predict(S)
predR = np.squeeze(output)
predR = predR[i1:i1+I.shape[1], j1:j1+I.shape[2], k1:k1+I.shape[3]]
pred = predR + im1[0:predR.shape[0],0:predR.shape[1],0:predR.shape[2]]
pred = minimum + spread * pred


In [12]:
# before saving, we go back to the original axis rearrangement, 
# in case your viewer does not handle RAS coordinates properly 
pred_rearranged = align_volume_to_ref(pred, aff, aff_ref=aff_orig)
utils.save_volume(pred_rearranged,aff_orig,None,output_file)
print('Visualize output with:  freeview ' + T1 + ' ' + T2 + ' ' + output_file)
print('(or using your favorite viewer!)')


Visualize output with:  freeview /tmp/T1.1mm.nii.gz /tmp/T2.1mm.nii.gz /tmp/output.nii.gz
(or using your favorite viewer!)
